1. (2 punto) Genera una base de datos con SQL Lite. En lugar de SQL Lite puedes utilizar

In [2]:
import sqlite3

conn = sqlite3.connect('big_bang_theory.db')
print("Base de datos creada y conectada con éxito")

Base de datos creada y conectada con éxito


2 punto Conecta a esa base de datos con pyodbc o SQL Alchemy. Crea una tabla en la base de datos desde un data frame de pandas creado a partir de los datos del fichero big_bang_theory_dataset.csv que hay en el aula virtual.

In [7]:
import pandas as pd
from sqlalchemy import create_engine
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the absolute path to the CSV file
csv_file_path = os.path.join(current_dir, 'big_bang_theory_dataset.csv')

# Cargar el dataset en un DataFrame using the absolute path
df = pd.read_csv(csv_file_path)

# Crear la conexión a la base de datos con SQLAlchemy
engine = create_engine('sqlite:///big_bang_theory.db')

# Crear la tabla en la base de datos
df.to_sql('big_bang_theory', con=engine, if_exists='replace', index=False)
print("Tabla creada en la base de datos con éxito.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/big_bang_theory_dataset.csv'

3. Realiza un histograma con las localizaciones donde Sheldon dice exclusivamente “Penny.”. Representa junto al anterior histograma otro con las localizaciones donde Sheldon dice algún texto que contenga “Penny” (podrías utilizar el método de pandas Series.str.contains).

In [ ]:
import matplotlib.pyplot as plt

# Filtrar los diálogos donde Sheldon dice exclusivamente "Penny."
df_penny_exact = df[(df['Personaje'] == 'Sheldon') & (df['Texto'] == "Penny.")]
# Filtrar los diálogos donde Sheldon menciona "Penny" en cualquier parte del texto
df_penny_contains = df[(df['Personaje'] == 'Sheldon') & (df['Texto'].str.contains("Penny", case=False))]

# Crear histogramas
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
df_penny_exact['Localización'].value_counts().plot(kind='bar', color='blue')
plt.title("Localizaciones donde Sheldon dice exclusivamente 'Penny.'")
plt.xlabel("Localización")
plt.ylabel("Frecuencia")

plt.subplot(1, 2, 2)
df_penny_contains['Localización'].value_counts().plot(kind='bar', color='orange')
plt.title("Localizaciones donde Sheldon menciona 'Penny'")
plt.xlabel("Localización")
plt.ylabel("Frecuencia")

plt.tight_layout()
plt.show()


4. Usando matplotlib. Representa las localizaciones de la escena en el eje Y y el personaje que habla en el eje X.

In [ ]:
plt.figure(figsize=(10, 8))

# Representar localizaciones en el eje Y y personajes en el eje X
plt.scatter(df['Personaje'], df['Localización'], alpha=0.5)
plt.title("Localizaciones de las escenas vs Personaje que habla")
plt.xlabel("Personaje")
plt.ylabel("Localización")
plt.xticks(rotation=90)
plt.show()


5. Agrupa el conjunto de datos por la temporada y el personaje. Para cada grupo genera una columna con el número de veces que se pronuncia la palabra “Penny” y otra columna con un índice para el grupo creado por personaje y temporada (podrías utilizar el método de pandas ngroup). Usando sklearn, realiza una regresión logística que relacione el número de veces que se dice “Penny” en una temporada con el personaje que lo dice


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Contar el número de veces que cada personaje menciona "Penny" por temporada
df['Penny_count'] = df['Texto'].str.contains("Penny", case=False).astype(int)
grouped = df.groupby(['Temporada', 'Personaje']).agg(
    Penny_count=('Penny_count', 'sum')
).reset_index()

# Crear índice para cada grupo por personaje y temporada
grouped['Personaje_index'] = grouped.groupby('Personaje').ngroup()

# Preparar los datos para la regresión logística
X = grouped[['Penny_count']]
y = grouped['Personaje_index']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Realizar predicciones y evaluar el modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
